In [26]:
import os
import numpy as np
import pandas as pd
import nltk
import torch
from transformers import BertTokenizer
from torchtext.legacy import data
from sklearn.model_selection import train_test_split
from transformers import BertModel
import transformers
import torch.nn as nn
import math
import time
import pickle

#from transformers import *
import torch.optim as optim

In [2]:
abspath = os.path.abspath('')
dname = os.path.dirname(abspath)
os.chdir(dname)
print(dname)

c:\Users\wongy\OneDrive\Desktop\duplicate-questions-pair-detection


In [27]:
train_df = pd.read_feather('data/processed/train_dataset.feather')
test_df = pd.read_feather('data/processed/test_dataset.feather')
val_df = pd.read_feather('data/processed/validation_dataset.feather')

In [29]:
val_df.head()

,index,qid1,qid2,question1,q1_cleaned,q2_cleaned,q1_trimmed,q2_trimmed,q1_start,q2_start,length_diff,lc_substring,lc_subsequence,jaccard_dist,common_words,common_ratio,levenshtein,fuzz_qratio,fuzz_wratio,is_duplicate
0,400250,14340,533599,What movie website can I watch movies on witho...,what movie website can i watch movie on withou...,how can i watch movie online without a credit ...,what movie website can i watch movie on withou...,how can i watch movie online without a credit ...,what,how,21,21,44,0.375000,7,0.057851,0.727273,73,81,1
1,67108,48332,116190,Is teleportation real?,is teleportation real,is teleportation possible,is teleportation real,is teleportation possible,is,is,-4,17,18,0.520000,2,0.043478,0.782609,78,82,0
2,174821,269320,269321,What do foreigners think about China?,what do foreigner think about china,what do the most foreigner think about china,what do foreigner think about china,what do the most foreigner think about china,what,what,-9,28,35,0.651163,6,0.075949,0.886076,89,95,1
3,398717,531964,531965,Is Recep Tayyip Erdogan the greatest leader of...,is recep tayyip erdogan the greatest leader of...,why am i expected to do something in life,is recep tayyip erdogan the greatest leader of...,why am i expected to do something in life,is,why,24,2,20,0.000000,0,0.000000,0.377358,38,38,0
4,187438,46329,42009,How do I improve my communication skills.?,how do i improve my communication skill,how can i improve my communication skill,how do i improve my communication skill,how can i improve my communication skill,how,how,0,33,37,0.690476,6,0.075949,0.936709,94,94,1


In [4]:
# def trim_sentence(sent):
#     try:
#         sent = sent.split()
#         sent = sent[:128]
#         return " ".join(sent)
#     except:
#         return sent

# train_df['q1_trimmed'] = train_df['q1_cleaned'].apply(lambda x: trim_sentence(x))
# train_df['q2_trimmed'] = train_df['q2_cleaned'].apply(lambda x: trim_sentence(x))

# val_df['q1_trimmed'] = val_df['q1_cleaned'].apply(lambda x: trim_sentence(x))
# val_df['q2_trimmed'] = val_df['q2_cleaned'].apply(lambda x: trim_sentence(x))

# test_df['q1_trimmed'] = test_df['q1_cleaned'].apply(lambda x: trim_sentence(x))
# test_df['q2_trimmed'] = test_df['q2_cleaned'].apply(lambda x: trim_sentence(x))

In [5]:
# def common_words(row):
#     w1 = set(map(lambda word: word.lower().strip(), row['q1_cleaned'].split(" ")))
#     w2 = set(map(lambda word: word.lower().strip(), row['q2_cleaned'].split(" ")))    
#     return len(w1 & w2)
    
# df['jaccard_dist'] = nltk.jaccard_distance(set(df['q1_cleaned']), set(df['q2_cleaned']))
# df['common_words'] = df.apply(common_words, axis=1)
# df['common_ratio'] = df.apply(lambda row: row['common_words'] / (len(row['q1_cleaned']) + len(row['q2_cleaned'])), axis=1)

In [6]:
# Masking the tokens to feed into BERT
def sent1_token_type(sentence):
    try:
        return [0]* len(sentence)
    except:
        return []
    
#Get list of 1s
def sent2_token_type(sentence):
    try:
        return [1]* len(sentence)
    except:
        return []

#combine from lists
def combine_seq(seq):
    return " ".join(seq)

#combines from lists of int
def combine_mask(mask):
    mask = list(map(str, mask))
    return " ".join(mask)

#convert attention mask back to list of int
def convert_mask(tok_ids):
    tok_ids = [int(x) for x in tok_ids]
    return tok_ids

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
cls_token = tokenizer.cls_token
sep_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token
cls_token_idx = tokenizer.cls_token_id
sep_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

def tokenize_bert(sentence):
    tokens = tokenizer.tokenize(sentence) 
    return tokens


def split_and_cut(sentence):
    tokens = sentence.strip().split(" ")
    tokens = tokens[:max_input_length] # make sure that it does not overflow
    return tokens

In [30]:
train_df['q1_padded'] = '[CLS] ' + train_df['q1_trimmed'] + ' [SEP] '
train_df['q2_padded'] = train_df['q2_trimmed'] + ' [SEP]'

train_df['q1_bert_tokens'] = train_df['q1_padded'].apply(lambda x: tokenize_bert(x))
train_df['q2_bert_tokens'] = train_df['q2_padded'].apply(lambda x: tokenize_bert(x))

val_df['q1_padded'] = '[CLS] ' + val_df['q1_trimmed'] + ' [SEP] '
val_df['q2_padded'] = val_df['q2_trimmed'] + ' [SEP]'

val_df['q1_bert_tokens'] = val_df['q1_padded'].apply(lambda x: tokenize_bert(x))
val_df['q2_bert_tokens'] = val_df['q2_padded'].apply(lambda x: tokenize_bert(x))

test_df['q1_padded'] = '[CLS] ' + test_df['q1_trimmed'] + ' [SEP] '
test_df['q2_padded'] = test_df['q2_trimmed'] + ' [SEP]'

test_df['q1_bert_tokens'] = test_df['q1_padded'].apply(lambda x: tokenize_bert(x))
test_df['q2_bert_tokens'] = test_df['q2_padded'].apply(lambda x: tokenize_bert(x))

In [32]:
train_df['q1_token_type'] = train_df['q1_bert_tokens'].apply(lambda x: sent1_token_type(x))
train_df['q2_token_type'] = train_df['q2_bert_tokens'].apply(lambda x: sent2_token_type(x))

val_df['q1_token_type'] = val_df['q1_bert_tokens'].apply(lambda x: sent1_token_type(x))
val_df['q2_token_type'] = val_df['q2_bert_tokens'].apply(lambda x: sent2_token_type(x))

test_df['q1_token_type'] = test_df['q1_bert_tokens'].apply(lambda x: sent1_token_type(x))
test_df['q2_token_type'] = test_df['q2_bert_tokens'].apply(lambda x: sent2_token_type(x))

In [33]:
train_df['sequence'] = train_df['q1_bert_tokens'] + train_df['q2_bert_tokens']
train_df['attn_mask'] = train_df['sequence'].apply(lambda x: sent2_token_type(x)) # every word needs attention
train_df['token_type'] = train_df['q1_token_type'] + train_df['q2_token_type']

val_df['sequence'] = val_df['q1_bert_tokens'] + val_df['q2_bert_tokens']
val_df['attn_mask'] = val_df['sequence'].apply(lambda x: sent2_token_type(x)) # every word needs attention
val_df['token_type'] = val_df['q1_token_type'] + val_df['q2_token_type']

test_df['sequence'] = test_df['q1_bert_tokens'] + test_df['q2_bert_tokens']
test_df['attn_mask'] = test_df['sequence'].apply(lambda x: sent2_token_type(x)) # every word needs attention
test_df['token_type'] = test_df['q1_token_type'] + test_df['q2_token_type']

In [35]:
# Make all the inputs to be sequential in string instead of list
train_df['sequence'] = train_df['sequence'].apply(lambda x: combine_seq(x))
train_df['attn_mask'] = train_df['attn_mask'].apply(lambda x: combine_mask(x))
train_df['token_type'] = train_df['token_type'].apply(lambda x: combine_mask(x))

val_df['sequence'] = val_df['sequence'].apply(lambda x: combine_seq(x))
val_df['attn_mask'] = val_df['attn_mask'].apply(lambda x: combine_mask(x))
val_df['token_type'] = val_df['token_type'].apply(lambda x: combine_mask(x))

test_df['sequence'] = test_df['sequence'].apply(lambda x: combine_seq(x))
test_df['attn_mask'] = test_df['attn_mask'].apply(lambda x: combine_mask(x))
test_df['token_type'] = test_df['token_type'].apply(lambda x: combine_mask(x))

In [36]:
#For sequence
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = split_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)
#For label
LABEL = data.LabelField()

#For Attention mask
ATTENTION = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = split_and_cut,
                  preprocessing = convert_mask,
                  pad_token = pad_token_idx)
#For token type ids
TTYPE = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = split_and_cut,
                  preprocessing = convert_mask,
                  pad_token = 1)

In [37]:
train_df = train_df[['sequence', 'attn_mask', 'token_type', 'is_duplicate']]
val_df = val_df[['sequence', 'attn_mask', 'token_type', 'is_duplicate']]
test_df = test_df[['sequence', 'attn_mask', 'token_type']]

In [39]:
train_df.to_csv('data/processed/bert_train.csv', index = False)
val_df.to_csv('data/processed/bert_val.csv', index = False)
test_df.to_csv('data/processed/bert_test.csv', index = False)

In [40]:
fields = [('sequence', TEXT), ('attn_mask', ATTENTION), ('token_type', TTYPE), ('is_duplicate', LABEL)]

train_data, valid_data, test_data = data.TabularDataset.splits(path = 'data/processed/',
                                                    train = 'bert_train.csv',
                                                    validation = 'bert_val.csv',
                                                    test = 'bert_test.csv',
                                                    format = 'csv',
                                                    fields = fields,
                                                    skip_header = True)
#Create iterator
BATCH_SIZE = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [41]:
train_iterator, valid_iterator, test_iterator= data.BucketIterator.splits((train_data, valid_data, test_data), 
                                                            batch_size = BATCH_SIZE,
                                                            sort_key = lambda x: len(x.sequence),
                                                            sort_within_batch = False, 
                                                            device = device)

In [42]:
LABEL.build_vocab(train_data)
print(len(LABEL.vocab))

2


In [43]:
bert_model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
class BERTNLIModel(nn.Module):
    def __init__(self, bert_model, output_dim):
        super().__init__()
        self.bert = bert_model
        embedding_dim = bert_model.config.to_dict()['hidden_size']
        self.out = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, sequence, attn_mask, token_type):
        embedded = self.bert(input_ids=sequence, attention_mask=attn_mask, token_type_ids=token_type)[1]
        output = self.out(embedded)
        return output

OUTPUT_DIM = len(LABEL.vocab)
model = BERTNLIModel(bert_model, OUTPUT_DIM).to(device)

In [45]:
optimizer = transformers.AdamW(model.parameters(),lr=2e-5,eps=1e-6,correct_bias=False)
criterion = nn.CrossEntropyLoss().to(device)
def get_scheduler(optimizer, warmup_steps):
    scheduler = transformers.get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)
    return scheduler

c:\Users\wongy\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [47]:
def categorical_accuracy(preds, y):
    max_preds = preds.argmax(dim = 1, keepdim = True)

    correct = (max_preds.squeeze(1)==y).float()

    return correct.sum() / len(y)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def train(model, iterator, optimizer, criterion, scheduler):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad() # clear gradients first
        torch.cuda.empty_cache() # releases all unoccupied cached memory
        
        sequence = batch.sequence
        attn_mask = batch.attn_mask
        token_type = batch.token_type
        label = batch.is_duplicate
        
        predictions = model(sequence, attn_mask, token_type)
        loss = criterion(predictions, label)
        acc = categorical_accuracy(predictions, label)
        
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()

    with torch.no_grad():
        for batch in iterator:
            sequence = batch.sequence

            attn_mask = batch.attn_mask
            token_type = batch.token_type
            labels = batch.is_duplicate

            predictions = model(sequence, attn_mask, token_type)
            loss = criterion(predictions, labels)
            acc = categorical_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
N_EPOCHS = 10
warmup_percent = 0.2
total_steps = math.ceil(N_EPOCHS*len(train_data)*1./BATCH_SIZE)
warmup_steps = int(total_steps*warmup_percent)
scheduler = get_scheduler(optimizer, warmup_steps)
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, scheduler)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'bert_based_quora_model.pt')
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')

KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load('src/bert_based_quora_model.pt'))
val_loss, val_acc = evaluate(model, valid_iterator, criterion)

In [ ]:
import pickle
filename = 'bert_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
def predict_inference(q1, q2, model, device):
    model.eval()
    
    q1 = '[CLS] ' + str(q1) + ' [SEP]'
    q2 = str(q2) + ' [SEP]'
    
    q1_t = tokenize_bert(q1)
    q2_t = tokenize_bert(q2)
    
    q1_type = sent1_token_type(q1_t)
    q2_type = sent2_token_type(q2_t)
    
    indexes = q1_t + q2_t
    indexes = tokenizer.convert_tokens_to_ids(indexes)
    
    indexes_type = q1_type + q2_type
    
    attn_mask = sent2_token_type(indexes)
    
    indexes = torch.LongTensor(indexes).unsqueeze(0).to(device)
    indexes_type = torch.LongTensor(indexes_type).unsqueeze(0).to(device)
    attn_mask = torch.LongTensor(attn_mask).unsqueeze(0).to(device)
    
    prediction = model(indexes, attn_mask, indexes_type)
    # prediction = prediction.argmax(dim=-1).item()
    return prediction